# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,295.171104,1.561888,0.497081,0.092877,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.980251,0.970350,0.966757,0.968553,0.970001,0.005527,1
5,446.761299,3.911945,0.293494,0.021662,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.976661,0.968553,0.969452,0.968553,0.969642,0.003829,2
3,434.379192,1.883536,0.575126,0.233027,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.975763,0.965858,0.970350,0.967655,0.969103,0.003711,3
0,199.310982,0.548369,0.402546,0.032887,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.961400,0.977558,0.971249,0.968553,0.965858,0.968924,0.005406,4
4,349.976348,17.025231,0.425998,0.102087,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.973070,0.968553,0.969452,0.965858,0.968385,0.002864,5


In [4]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,423.893482,3.934451,0.421817,0.054064,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.976661,0.968553,0.966757,0.970350,0.969462,0.004019,1
8,402.117246,44.783517,0.300927,0.083109,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.978456,0.967655,0.962264,0.973046,0.968923,0.006109,2
4,278.884414,1.531584,0.408779,0.038088,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964093,0.975763,0.969452,0.965858,0.967655,0.968564,0.004019,3
2,277.124966,2.093588,0.374306,0.017822,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.969452,0.964061,0.966757,0.968564,0.005332,4
1,173.717714,2.596950,0.345005,0.023134,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964991,0.979354,0.968553,0.962264,0.967655,0.968563,0.005826,5
